In [1]:
using POMDPs
using Random # for AbstractRNG
using POMDPModelTools
using Pkg

using DataFrames
Pkg.add("JSON")

POMDPs.add_registry()
using Pkg; Pkg.add("DiscreteValueIteration")

using DiscreteValueIteration

  Updating registry at `~/.julia/registries/General`
  Updating git-repo `https://github.com/JuliaRegistries/General.git`
Fetching: [========================================>]  100.0 %.0 %  Updating registry at `~/.julia/registries/JuliaPOMDP`
  Updating git-repo `https://github.com/JuliaPOMDP/Registry`
 Resolving package versions...
  Updating `~/.julia/environments/v1.0/Project.toml`
 [no changes]
  Updating `~/.julia/environments/v1.0/Manifest.toml`
 [no changes]
   Cloning registry from "https://github.com/JuliaPOMDP/Registry"
Fetching: [========================================>]  100.0 %.0 %=========>                               ]  21.4 %]  42.7 %==========================>              ]  64.1 %]  81.6 %

┌ Info: registry `JuliaPOMDP` already exists in `~/.julia/registries`
└ @ POMDPs /Users/efan/.julia/packages/POMDPs/JiYXY/src/utils.jl:65


 Resolving package versions...
  Updating `~/.julia/environments/v1.0/Project.toml`
 [no changes]
  Updating `~/.julia/environments/v1.0/Manifest.toml`
 [no changes]


In [2]:
using Pkg
Pkg.add("StaticArrays")
using POMDPSimulators
using POMDPPolicies

 Resolving package versions...
  Updating `~/.julia/environments/v1.0/Project.toml`
 [no changes]
  Updating `~/.julia/environments/v1.0/Manifest.toml`
 [no changes]


In [ ]:

include("./ParallelChains.jl")

In [ ]:
num_chains = 5
num_states = 6
r_mean = 5.0
r_std = 2.0

true_values = DataFrame(theta = [r_mean for i in 1:num_chains],
                        std = [r_std  for i in 1:num_chains])

In [ ]:
using POMDPSimulators
using POMDPPolicies
rng = MersenneTwister(1235);

PParallelChainMDP() = PParallelChainMDP(num_states+1,num_chains, .9,

        true_values.theta,
        true_values.std,
        MersenneTwister(1235))
m = PParallelChainMDP()

# policy that maps every input to an action
policy = FunctionPolicy(s->3)
for i in 1:3
    for (s, a, r) in stepthrough(m, policy, "s,a,r", max_steps=10)
      render(m, (s,a,r))
      println("s,a,r:($s,$a,$r)")
    end
end

In [ ]:
curry(f, a) = (xs...) -> f(a, xs...)

In [ ]:
using POMDPSimulators
using POMDPPolicies
rng = MersenneTwister(1235);

num_chains = 10
num_states = 30
r_mean = 5.0
r_std = 2.0
@requirements_info ValueIterationSolver() PParallelChainMDP(num_states+1,num_chains, .9,

        randn(rng, Float32, num_chains) .* r_std .+ r_mean)

In [ ]:
num_chains=2
r_mean = 0
r_std = 1
## sigma is 1
## Reward is noisy ~ N(theta_c, sigma^2)
## Initial prior mean is N(theta_c, sigma^2 + c)
rng = MersenneTwister(1235)
th_c = [randn(rng, Float64, 1)[1] * (100 + i) for i in 1:num_chains]
println(th_c)
true_values = DataFrame(theta = th_c,
                         std = [r_std  for i in 1:num_chains])
mdp = PParallelChainMDP(num_states+1,num_chains, .9,
        true_values.theta,
        true_values.std, 
        rng)
# @requirements_info ValueIterationSolver() mdp
solver = ValueIterationSolver(max_iterations=100, belres=1e-6, include_Q=true, verbose=true) # initializes the Solver type
vip = solve(solver, mdp) # runs value iterations


In [ ]:
vip.qmat

In [ ]:
LinearIndices((num_chains, num_states))[1,1]

In [ ]:

for i in 1:num_chains
    println("q value for (1,1),$i is $(vip.qmat[LinearIndices((num_chains, num_states))[1,1],i])")
    println("q value for $((i,num_states)),1 is $(vip.qmat[LinearIndices((num_chains, num_states))[i,num_states],1])")
end

In [ ]:
mdp.Rs

In [ ]:
using POMDPSimulators
using POMDPPolicies



for (s, a, r) in stepthrough(mdp, vip, "s,a,r", max_steps=40)
    render(mdp, (s,a,r))
    println("s,a,r:($s,$a,$r)")
end



In [3]:
function ucb_pol(li, policy, priors, i, actions, s)
    best_action = action(policy, s)
    #println("in state: $s, best action:$best_action")
    return action(policy, s)
end

ucb_pol (generic function with 1 method)

In [4]:
curry(f, x) = (xs...) -> f(x, xs...)
function ucb_mdp_builder(rng, true_mdp, priors, i, num_states, num_chains, steps, start_state)
  # build mdp
  new_means = priors.theta + priors.std
  mdp = PParallelChainMDP(num_states+1,num_chains, .9,
        [priors.theta[j] + 4* priors.std[j] for j in 1:num_chains])
  solver = ValueIterationSolver(max_iterations=1000, belres=1e-6, include_Q=true)#, verbose=true) # initializes the Solver type
  vip = solve(solver, mdp)
  li = LinearIndices((num_chains, num_states))
  new_policy = FunctionPolicy(curry(curry(curry(curry(curry(ucb_pol, li), vip), priors),i), actions))
  if start_state == nothing
    return Iterators.Stateful(stepthrough(deepcopy(true_mdp), new_policy, "s,a,r,sp,t",
                              max_steps=steps))
  else
     return Iterators.Stateful(stepthrough(deepcopy(true_mdp), new_policy, start_state, "s,a,r,sp,t",
                              max_steps=steps))       
  end
end

ucb_mdp_builder (generic function with 1 method)

In [47]:

include("./ParallelChains.jl")
using Distributions
function do_runs(nruns, epochs, num_agents, num_chains, num_states, H, update_priors_check, update_priors,
                 mdp_iter_builder, is_thompson_sampling, base, rng)
    # UCB

    # setup constants here

    # So best idea:
    # for ucb:
    # we generate an mdp with mu + sigma rewards (priors) for each end node.
    #  we collect history until the criteria in PAC-EXPLORE then update the prior
    # 
    # for thompson sampling
    # at each time step (might skip this for this problem and save for max rew path) 
    #   take rewards and use to create a posterior.
    # for seed sampling
    # at beginning of episode
    #   each agent generates a new random "seed" 
    #   at each time step
    #     each agent generates a new mdp based on a deterministic mapping from seed to rewards (which also takes
    #              history into account)
    # this needs to update the actual reward.

    ## PC notes
    ## theta_c ~ N(0, 100 + c)
    r_mean = 0
    r_std = 1
    ## sigma is 1
    ## Reward is noisy ~ N(theta_c, sigma^2)
    ## Initial prior mean is N(theta_c, sigma^2 + c)
    runs = []
    true_values_list = DataFrame( run = [],
                                 c = [],
                                 theta = [],
                                 std = [])
    for run in 1:nruns
        th_c = [randn(rng, Float64, 1)[1] * sqrt(base + i) for i in 1:num_chains]
        #th_c[1] = 300
        println(th_c)
        th_c_sd = sqrt(var(th_c))
        r_std = th_c_sd
        true_values = DataFrame( run = [run for i in 1:num_chains],
                                 c = [i for i in 1:num_chains],
                                 theta = th_c,
                                 std = [r_std  for i in 1:num_chains])
        print("true mdp: $true_values")
        #priors = DataFrame(theta = [th_c[i] for i in 1:num_chains],
        priors = DataFrame(theta = [0 for i in 1:num_chains],
                           std = [(sqrt(r_std^2) + i) for i in 1:num_chains],
                           state = [(chain,num_states) for chain in 1:num_chains])
        print("priors: $priors")
        mdp = PParallelChainMDP(num_states+1,num_chains, .9,
                true_values.theta)
        do_update_priors() =  false
        # priors = randn(rng, Float32, num_chains) .* (r_std + c) .+ r_mean
        hist = run_chain!(
                   mdp_iter_builder=mdp_iter_builder,
                   true_mdp=mdp,
                   do_update_priors=update_priors_check,
                   update_priors=update_priors,
                   priors=priors,
                   true_vals=true_values,
                   n_agents=num_agents,
                   num_states=num_states,
                   num_chains=num_chains,
                   epochs=epochs,
                   steps=H,
                   is_thompson_sampling=is_thompson_sampling,
                   rng=rng) #,
                   #rev_action_map=rev_action_map)
        #print(Q_tables)
        println("(e,i,t,st,r)")
        hist
        for (e, ag, t, st, r)  in hist
          push!(runs, (run, e, ag, t, st, r)) 
        end
        append!(true_values_list, true_values)

    end
    return (true_values_list, runs)
end

do_runs (generic function with 1 method)

In [ ]:
num_agents = 100
num_chains = 10
num_states = 10
epochs = 1
nruns = 1
H = 30
do_update_priors() = true
true_values_list, runs = do_runs(nruns, 1, num_agents, num_chains, num_states, H, do_update_priors,
                                 ucb_update_priors, ucb_mdp_builder, false, 100)
true_values_list, runs

chain_len = num_states
num_chains = num_chains
ucb_regret = get_average_regret(runs, chain_len, num_chains, true_values_list)

In [29]:

using DataFrames
using Statistics
function ucb_update_priors(priors, hist, true_vals)
    # each state must be visted me times to update the prior
    # me is # of S*D^2 s= states, D =diameter (https://www.aaai.org/ocs/index.php/AAAI/AAAI15/paper/view/9542/9919)
    #     (D defined in http://www.jmlr.org/papers/volume11/jaksch10a/jaksch10a.pdf)
    ## ^^^ Actually, think should just do this statistically, and if n > 2, then we can adjust the prior.
    df = DataFrame(epoch = [x[1] for x in hist], agent=[x[2] for x in hist],
               time = [x[3] for x in hist], chain = [x[4][1] for x in hist], chain_state = [x[4][2] for x in hist],
        reward = [x[5] for x in hist])
    new_theta = []
    new_std = []
    new_state = []
    for arow in eachrow(priors)
        if nrow(df[(df.chain.==arow.state[1]).&(df.chain_state.==arow.state[2]),:]) > 1
            mu_p = arow.theta
            true_std = true_vals[priors.Row,:].std[1]
            s_p_2 = arow.std^2
            mu_s = mean(df[(df.chain.==arow.state[1]).&(df.chain_state.==arow.state[2]),:].reward)
            s_s_2 = var(df[(df.chain.==arow.state[1]).&(df.chain_state.==arow.state[2]),:].reward)
            cnt = nrow(df[(df.chain.==arow.state[1]).&(df.chain_state.==arow.state[2]),:])
            #tmp_theta =  (s_p_2/(s_s_2 + s_p_2)) * mu_s + (s_s_2/(s_s_2 + s_p_2)) * mu_p
            #tmp_std = 1/(1/s_p_2 + 1/s_s_2)
            tmp_theta =  (s_p_2/(true_std/cnt + s_p_2)) * mu_s + (s_s_2/(true_std/cnt + s_p_2)) * mu_p
            tmp_std = 1/(1/s_p_2 + cnt/true_std)
        else
            tmp_theta =  arow.theta
            tmp_std = arow.std
        end
        push!(new_state, arow.state)
        push!(new_theta, tmp_theta)
        push!(new_std, tmp_std)
    end
    new_priors = DataFrame(theta=new_theta, std=new_std, state=new_state)
    return new_priors
end



ucb_update_priors (generic function with 2 methods)

In [ ]:
println(priors)
ucb_update_priors(priors, hist)

In [7]:
function get_average_regret(results, chain_len, num_chains, theta)
    df = DataFrame(run = [x[1] for x in results], epoch = [x[2] for x in results], agent=[x[3] for x in results],
               time = [x[4] for x in results], c = [x[5][1] for x in results], state = [x[5][2] for x in results],
               reward = [x[6] for x in results])
    df_R = df[df.reward.!==0,:]
    th_max = by(theta, :run, :theta => maximum)
    th_min = by(theta, :run, :theta => minimum)
    theta_joined = join(join(theta, th_max, on = :run), th_min, on = :run)
    df_joined = join(df_R, theta_joined, on = [:run,:c] )
    # needs to be max theta
    df_joined.Regret = df_joined.theta_maximum - df_joined.reward
    df_joined
end

get_average_regret (generic function with 1 method)

In [ ]:
true_values_list

In [ ]:
# Thompson sampling
num_agents = 100
num_chains = 10
num_states = 10
epochs = 1
nruns = 1
H = 30
do_update_priors() = true
true_values_list, runs = do_runs(nruns, 1, num_agents, num_chains, num_states, H, do_update_priors,
           ucb_update_priors, ucb_mdp_builder, true, 100)
true_values_list, runs

chain_len = num_states
num_chains = num_chains
thompson_regret = get_average_regret(runs, chain_len, num_chains, true_values_list)

In [18]:
curry(f, x) = (xs...) -> f(x, xs...)
function ssgn_mdp_builder(rng, true_mdp, priors, i, num_states, num_chains, steps, start_state)
  # build mdp
  # new_means = priors.theta + priors.std
  mdp = PParallelChainMDP(num_states+1,num_chains, .9,
        #[randn(rng, Float32,1)[1] *  1 + priors.theta[j] for j in 1:num_chains])
        [randn(rng, Float32,1)[1] *  (2*priors.std[j]) + priors.theta[j] for j in 1:num_chains])
  solver = ValueIterationSolver(max_iterations=1000, belres=1e-6, include_Q=true)#, verbose=true) # initializes the Solver type
  vip = solve(solver, mdp)
  li = LinearIndices((num_chains, num_states))
  new_policy = FunctionPolicy(curry(curry(curry(curry(curry(ucb_pol, li), vip), priors),i), actions))
  if start_state == nothing
    return Iterators.Stateful(stepthrough(deepcopy(true_mdp), new_policy, "s,a,r,sp,t",
                              max_steps=steps))
  else
     return Iterators.Stateful(stepthrough(deepcopy(true_mdp), new_policy, start_state, "s,a,r,sp,t",
                              max_steps=steps))       
  end
end

ssgn_mdp_builder (generic function with 1 method)

In [ ]:
# Gaussian normal sampling
num_agents = 100
num_chains = 10
num_states = 10
epochs = 1
nruns = 1
H = 30
do_update_priors() = true
true_values_list, runs = do_runs(nruns, 1, num_agents, num_chains, num_states, H, do_update_priors,
    ucb_update_priors, ssgn_mdp_builder, false, 100)
true_values_list, runs

In [ ]:

chain_len = num_states
num_chains = num_chains
seed_regret = get_average_regret(runs, chain_len, num_chains, true_values_list)

In [ ]:
thompson_regret.name = ["Thompson Sampling" for i in 1:nrow(thompson_regret)]

In [ ]:
seed_regret.name = ["Seed Sampling" for i in 1:nrow(seed_regret)]

In [ ]:
ucb_regret.name = ["UCB" for i in 1:nrow(ucb_regret)]

In [ ]:
using CSV
all_regret = deepcopy(thompson_regret)
append!(all_regret,seed_regret)
append!(all_regret,ucb_regret)
CSV.write("pc_simulation.csv", all_regret)
all_regret

In [ ]:

include("./ParallelChains.jl")
# Full loop
full_df = nothing
base = 100

rng = MersenneTwister()
for num_agents in [1,10,100,1000,10000]
    println("*****************number of agents:$num_agents*******************")
    num_chains = 10
    num_states = 5
    epochs = 1
    nruns = 30
    H = 12
    do_update_priors() = true
    for run_type in ["Thompson Sampling", "Seed Sampling", "UCB"]
        println("------------------Run Type:$run_type------------------------")
        if run_type == "Thompson Sampling"
            true_values_list, runs = do_runs(nruns, epochs, num_agents, num_chains, num_states, H,
                                             do_update_priors, ucb_update_priors,  ssgn_mdp_builder, true,
                                             base, rng)
        elseif run_type == "UCB"
            true_values_list, runs = do_runs(nruns, epochs, num_agents, num_chains, num_states, H,
                                             do_update_priors, ucb_update_priors, ucb_mdp_builder, false,
                                             base, rng)
        else
            true_values_list, runs = do_runs(nruns, epochs, num_agents, num_chains, num_states, H,
                                             do_update_priors, ucb_update_priors,  ssgn_mdp_builder, false,
                                             base, rng)

        end

        chain_len = num_states
        num_chains = num_chains
        regret = get_average_regret(runs, chain_len, num_chains, true_values_list)
        regret.name = [run_type for i in 1:nrow(regret)]
        regret.num_agents = [num_agents for i in 1:nrow(regret)]
        regret.base = [base for i in 1:nrow(regret)]
        if full_df == nothing
            full_df = regret
        else
            append!(full_df,regret)
        end
    end
end 

println("***********************Done!!!*********************************")





*****************number of agents:1*******************
------------------Run Type:Thompson Sampling------------------------
[7.09196, -3.98074, 3.64706, 3.62183, -11.3914, -13.7777, -3.09235, -10.8934, 8.12073, 20.3308]
true mdp: 10×4 DataFrame
│ Row │ run   │ c     │ theta    │ std     │
│     │ Int64 │ Int64 │ Float64  │ Float64 │
├─────┼───────┼───────┼──────────┼─────────┤
│ 1   │ 1     │ 1     │ 7.09196  │ 10.644  │
│ 2   │ 1     │ 2     │ -3.98074 │ 10.644  │
│ 3   │ 1     │ 3     │ 3.64706  │ 10.644  │
│ 4   │ 1     │ 4     │ 3.62183  │ 10.644  │
│ 5   │ 1     │ 5     │ -11.3914 │ 10.644  │
│ 6   │ 1     │ 6     │ -13.7777 │ 10.644  │
│ 7   │ 1     │ 7     │ -3.09235 │ 10.644  │
│ 8   │ 1     │ 8     │ -10.8934 │ 10.644  │
│ 9   │ 1     │ 9     │ 8.12073  │ 10.644  │
│ 10  │ 1     │ 10    │ 20.3308  │ 10.644  │priors: 10×3 DataFrame
│ Row │ theta │ std     │ state   │
│     │ Int64 │ Float64 │ Tuple…  │
├─────┼───────┼─────────┼─────────┤
│ 1   │ 0     │ 11.644  │ (1, 5)  │
│ 2 

e: 1, t: 1, agent 1, result: (s = [10, 2], a = 1, r = 0, sp = [10, 3], t = 1)
e: 1, t: 1, agent 1, result: (s = [10, 3], a = 1, r = 0, sp = [10, 4], t = 1)
e: 1, t: 1, agent 1, result: (s = [10, 4], a = 1, r = 0, sp = [10, 5], t = 1)
e: 1, t: 1, agent 1, result: (s = [10, 5], a = 1, r = -2.2187521853139454, sp = [10, 6], t = 1)
(e,i,t,st,r)
[3.03566, -11.526, -6.02606, 9.92215, 14.9466, -5.61987, -4.81223, -3.16036, -7.98114, -0.819159]
true mdp: 10×4 DataFrame
│ Row │ run   │ c     │ theta     │ std     │
│     │ Int64 │ Int64 │ Float64   │ Float64 │
├─────┼───────┼───────┼───────────┼─────────┤
│ 1   │ 7     │ 1     │ 3.03566   │ 8.26525 │
│ 2   │ 7     │ 2     │ -11.526   │ 8.26525 │
│ 3   │ 7     │ 3     │ -6.02606  │ 8.26525 │
│ 4   │ 7     │ 4     │ 9.92215   │ 8.26525 │
│ 5   │ 7     │ 5     │ 14.9466   │ 8.26525 │
│ 6   │ 7     │ 6     │ -5.61987  │ 8.26525 │
│ 7   │ 7     │ 7     │ -4.81223  │ 8.26525 │
│ 8   │ 7     │ 8     │ -3.16036  │ 8.26525 │
│ 9   │ 7     │ 9     │ -7.9

e: 1, t: 1, agent 1, result: (s = [5, 4], a = 1, r = 0, sp = [5, 5], t = 1)
e: 1, t: 1, agent 1, result: (s = [5, 5], a = 1, r = 7.230266101229605, sp = [5, 6], t = 1)
(e,i,t,st,r)
[6.31233, 2.35991, -17.2492, -7.58401, 9.41046, -4.88757, 4.53325, -3.31548, 6.57322, -0.954232]
true mdp: 10×4 DataFrame
│ Row │ run   │ c     │ theta     │ std     │
│     │ Int64 │ Int64 │ Float64   │ Float64 │
├─────┼───────┼───────┼───────────┼─────────┤
│ 1   │ 14    │ 1     │ 6.31233   │ 8.071   │
│ 2   │ 14    │ 2     │ 2.35991   │ 8.071   │
│ 3   │ 14    │ 3     │ -17.2492  │ 8.071   │
│ 4   │ 14    │ 4     │ -7.58401  │ 8.071   │
│ 5   │ 14    │ 5     │ 9.41046   │ 8.071   │
│ 6   │ 14    │ 6     │ -4.88757  │ 8.071   │
│ 7   │ 14    │ 7     │ 4.53325   │ 8.071   │
│ 8   │ 14    │ 8     │ -3.31548  │ 8.071   │
│ 9   │ 14    │ 9     │ 6.57322   │ 8.071   │
│ 10  │ 14    │ 10    │ -0.954232 │ 8.071   │priors: 10×3 DataFrame
│ Row │ theta │ std     │ state   │
│     │ Int64 │ Float64 │ Tuple…  │
├────

(e,i,t,st,r)
[4.17872, -14.0778, 8.8969, 2.37309, 3.11892, -7.6508, 0.364012, -15.0934, -10.1995, -12.4356]
true mdp: 10×4 DataFrame
│ Row │ run   │ c     │ theta    │ std     │
│     │ Int64 │ Int64 │ Float64  │ Float64 │
├─────┼───────┼───────┼──────────┼─────────┤
│ 1   │ 19    │ 1     │ 4.17872  │ 8.7628  │
│ 2   │ 19    │ 2     │ -14.0778 │ 8.7628  │
│ 3   │ 19    │ 3     │ 8.8969   │ 8.7628  │
│ 4   │ 19    │ 4     │ 2.37309  │ 8.7628  │
│ 5   │ 19    │ 5     │ 3.11892  │ 8.7628  │
│ 6   │ 19    │ 6     │ -7.6508  │ 8.7628  │
│ 7   │ 19    │ 7     │ 0.364012 │ 8.7628  │
│ 8   │ 19    │ 8     │ -15.0934 │ 8.7628  │
│ 9   │ 19    │ 9     │ -10.1995 │ 8.7628  │
│ 10  │ 19    │ 10    │ -12.4356 │ 8.7628  │priors: 10×3 DataFrame
│ Row │ theta │ std     │ state   │
│     │ Int64 │ Float64 │ Tuple…  │
├─────┼───────┼─────────┼─────────┤
│ 1   │ 0     │ 9.7628  │ (1, 5)  │
│ 2   │ 0     │ 10.7628 │ (2, 5)  │
│ 3   │ 0     │ 11.7628 │ (3, 5)  │
│ 4   │ 0     │ 12.7628 │ (4, 5)  │
│ 5   │ 

[2.79142, -5.19757, 17.4786, -2.77483, 5.77256, -10.0883, -16.0183, 4.34229, 6.2229, -15.7853]
true mdp: 10×4 DataFrame
│ Row │ run   │ c     │ theta    │ std     │
│     │ Int64 │ Int64 │ Float64  │ Float64 │
├─────┼───────┼───────┼──────────┼─────────┤
│ 1   │ 26    │ 1     │ 2.79142  │ 10.692  │
│ 2   │ 26    │ 2     │ -5.19757 │ 10.692  │
│ 3   │ 26    │ 3     │ 17.4786  │ 10.692  │
│ 4   │ 26    │ 4     │ -2.77483 │ 10.692  │
│ 5   │ 26    │ 5     │ 5.77256  │ 10.692  │
│ 6   │ 26    │ 6     │ -10.0883 │ 10.692  │
│ 7   │ 26    │ 7     │ -16.0183 │ 10.692  │
│ 8   │ 26    │ 8     │ 4.34229  │ 10.692  │
│ 9   │ 26    │ 9     │ 6.2229   │ 10.692  │
│ 10  │ 26    │ 10    │ -15.7853 │ 10.692  │priors: 10×3 DataFrame
│ Row │ theta │ std     │ state   │
│     │ Int64 │ Float64 │ Tuple…  │
├─────┼───────┼─────────┼─────────┤
│ 1   │ 0     │ 11.692  │ (1, 5)  │
│ 2   │ 0     │ 12.692  │ (2, 5)  │
│ 3   │ 0     │ 13.692  │ (3, 5)  │
│ 4   │ 0     │ 14.692  │ (4, 5)  │
│ 5   │ 0     │ 15.69

...

│ 10  │ 0     │ 22.6826 │ (10, 5) │e: 1, t: 1, agent 1, result: (s = [1, 1], a = 4, r = 0, sp = [4, 2], t = 1)
e: 1, t: 2, agent 1, result: (s = [4, 2], a = 1, r = 0, sp = [4, 3], t = 2)
e: 1, t: 1, agent 2, result: (s = [1, 1], a = 10, r = 0, sp = [10, 2], t = 1)
e: 1, t: 3, agent 1, result: (s = [4, 3], a = 1, r = 0, sp = [4, 4], t = 3)
e: 1, t: 2, agent 2, result: (s = [10, 2], a = 1, r = 0, sp = [10, 3], t = 2)
e: 1, t: 1, agent 3, result: (s = [1, 1], a = 9, r = 0, sp = [9, 2], t = 1)
e: 1, t: 1, agent 4, result: (s = [1, 1], a = 6, r = 0, sp = [6, 2], t = 1)
e: 1, t: 1, agent 5, result: (s = [1, 1], a = 8, r = 0, sp = [8, 2], t = 1)
e: 1, t: 4, agent 1, result: (s = [4, 4], a = 1, r = 0, sp = [4, 5], t = 4)
e: 1, t: 3, agent 2, result: (s = [10, 3], a = 1, r = 0, sp = [10, 4], t = 3)
e: 1, t: 2, agent 3, result: (s = [9, 2], a = 1, r = 0, sp = [9, 3], t = 2)
e: 1, t: 2, agent 4, result: (s = [6, 2], a = 1, r = 0, sp = [6, 3], t = 2)
e: 1, t: 2, agent 5, result: (s = [8, 2], a = 1

e: 1, t: 5, agent 10, result: (s = [1, 5], a = 1, r = -4.221099926579059, sp = [1, 6], t = 5)
(e,i,t,st,r)
[-1.68984, -6.34135, -11.3109, 7.97639, -3.1745, 0.229921, 4.38634, 11.1158, 11.4935, 3.67655]
true mdp: 10×4 DataFrame
│ Row │ run   │ c     │ theta    │ std     │
│     │ Int64 │ Int64 │ Float64  │ Float64 │
├─────┼───────┼───────┼──────────┼─────────┤
│ 1   │ 27    │ 1     │ -1.68984 │ 7.50113 │
│ 2   │ 27    │ 2     │ -6.34135 │ 7.50113 │
│ 3   │ 27    │ 3     │ -11.3109 │ 7.50113 │
│ 4   │ 27    │ 4     │ 7.97639  │ 7.50113 │
│ 5   │ 27    │ 5     │ -3.1745  │ 7.50113 │
│ 6   │ 27    │ 6     │ 0.229921 │ 7.50113 │
│ 7   │ 27    │ 7     │ 4.38634  │ 7.50113 │
│ 8   │ 27    │ 8     │ 11.1158  │ 7.50113 │
│ 9   │ 27    │ 9     │ 11.4935  │ 7.50113 │
│ 10  │ 27    │ 10    │ 3.67655  │ 7.50113 │priors: 10×3 DataFrame
│ Row │ theta │ std     │ state   │
│     │ Int64 │ Float64 │ Tuple…  │
├─────┼───────┼─────────┼─────────┤
│ 1   │ 0     │ 8.50113 │ (1, 5)  │
│ 2   │ 0     │ 9.5011

e: 1, t: 5, agent 6, result: (s = [8, 5], a = 1, r = 7.600209530539437, sp = [8, 6], t = 5)
e: 1, t: 5, agent 7, result: (s = [6, 5], a = 1, r = -8.99995103089938, sp = [6, 6], t = 5)
e: 1, t: 4, agent 8, result: (s = [6, 4], a = 1, r = 0, sp = [6, 5], t = 4)
e: 1, t: 4, agent 9, result: (s = [7, 4], a = 1, r = 0, sp = [7, 5], t = 4)
e: 1, t: 1, agent 10, result: (s = [1, 1], a = 10, r = 0, sp = [10, 2], t = 1)
e: 1, t: 5, agent 8, result: (s = [6, 5], a = 1, r = -8.99995103089938, sp = [6, 6], t = 5)
e: 1, t: 5, agent 9, result: (s = [7, 5], a = 1, r = -1.985318066956769, sp = [7, 6], t = 5)
e: 1, t: 2, agent 10, result: (s = [10, 2], a = 1, r = 0, sp = [10, 3], t = 2)
e: 1, t: 3, agent 10, result: (s = [10, 3], a = 1, r = 0, sp = [10, 4], t = 3)
e: 1, t: 4, agent 10, result: (s = [10, 4], a = 1, r = 0, sp = [10, 5], t = 4)
e: 1, t: 5, agent 10, result: (s = [10, 5], a = 1, r = 0.6913833527648966, sp = [10, 6], t = 5)
(e,i,t,st,r)
[0.123397, -11.9744, 3.00035, -11.547, 7.89756, -24.65

Excessive output truncated after 524378 bytes.

e: 1, t: 1, agent 9, result: (s = [1, 1], a = 9, r = 0, sp = [9, 2], t = 1)
e: 1, t: 1, agent 10, result: (s = [1, 1], a = 6, r = 0, sp = [6, 2], t = 1)
e: 1, t: 4, agent 4, result: (s = [1, 4], a = 1, r = 0, sp = [1, 5], t = 4)
e: 1, t: 3, agent 5, result: (s = [9, 3], a = 1, r = 0, sp = [9, 4], t = 3)
e: 1, t: 3, agent 6, result: (s = [4, 3], a = 1, r = 0, sp = [4, 4], t = 3)
e: 1, t: 3, agent 7, result: (s = [5, 3], a = 1, r = 0, sp = [5, 4], t = 3)
e: 1, t: 3, agent 8, result: (s = [4, 3], a = 1, r = 0, sp = [4, 4], t = 3)
e: 1, t: 2, agent 9, result: (s = [9, 2], a = 1, r = 0, sp = [9, 3], t = 2)
e: 1, t: 2, agent 10, result: (s = [6, 2], a = 1, r = 0, sp = [6, 3], t = 2)
e: 1, t: 5, agent 4, result: (s = [1, 5], a = 1, r = -12.087833446315713, sp = [1, 6], t = 5)
e: 1, t: 4, agent 5, result: (s = [9, 4], a = 1, r = 0, sp = [9, 5], t = 4)
e: 1, t: 4, agent 6, result: (s = [4, 4], a = 1, r = 0, sp = [4, 5], t = 4)
e: 1, t: 4, agent 7, result: (s = [5, 4], a = 1, r = 0, sp = [5, 5],

In [ ]:
using CSV
full_df[full_df[:Regret] .< 0.0,:Regret] = 0.0
CSV.write("pc_simulation.csv", full_df)
full_df

In [11]:

full_df[full_df.name.=="Seed Sampling",:]

,run,epoch,agent,time,c,state,reward,theta,std,theta_maximum,theta_minimum,Regret,name,num_agents,base
,Int64,Int64,Int64,Int64,Int64,Int64,Real,Any,Any,Float64,Float64,Float64,String,Int64,Int64
1,1,1,1,5,3,5,16.9001,17.1407,7.96621,17.1407,-8.4027,0.240627,Seed Sampling,10,100
2,1,1,2,5,8,5,5.76836,4.81265,7.96621,17.1407,-8.4027,11.3724,Seed Sampling,10,100
3,1,1,3,5,8,5,4.48508,4.81265,7.96621,17.1407,-8.4027,12.6556,Seed Sampling,10,100
4,1,1,4,5,8,5,4.94561,4.81265,7.96621,17.1407,-8.4027,12.1951,Seed Sampling,10,100
5,1,1,5,5,6,5,6.97628,7.07435,7.96621,17.1407,-8.4027,10.1644,Seed Sampling,10,100
6,1,1,6,5,8,5,4.45887,4.81265,7.96621,17.1407,-8.4027,12.6818,Seed Sampling,10,100
7,1,1,7,5,10,5,12.6464,13.3034,7.96621,17.1407,-8.4027,4.49435,Seed Sampling,10,100
8,1,1,8,5,10,5,14.2933,13.3034,7.96621,17.1407,-8.4027,2.84741,Seed Sampling,10,100
9,1,1,9,5,10,5,13.9089,13.3034,7.96621,17.1407,-8.4027,3.23177,Seed Sampling,10,100


In [65]:

full_df[(full_df.name.=="UCB").&(full_df.run.==1),:]

,run,epoch,agent,time,c,state,reward,theta,std,theta_maximum,theta_minimum,Regret,name,num_agents,base
,Int64,Int64,Int64,Int64,Int64,Int64,Real,Any,Any,Float64,Float64,Float64,String,Int64,Int64
1,1,1,1,5,10,5,-6.71911,-8.07105,12.9653,10.478,-25.7266,17.1971,UCB,1,100
2,1,1,1,5,10,5,8.19248,9.24593,8.45991,13.3922,-7.63904,5.19972,UCB,10,100
3,1,1,2,5,10,5,9.72549,9.24593,8.45991,13.3922,-7.63904,3.66671,UCB,10,100
4,1,1,3,5,10,5,9.67703,9.24593,8.45991,13.3922,-7.63904,3.71517,UCB,10,100
5,1,1,4,5,10,5,8.07192,9.24593,8.45991,13.3922,-7.63904,5.32028,UCB,10,100
6,1,1,5,5,10,5,10.475,9.24593,8.45991,13.3922,-7.63904,2.91721,UCB,10,100
7,1,1,6,5,10,5,9.73374,9.24593,8.45991,13.3922,-7.63904,3.65847,UCB,10,100
8,1,1,7,5,9,5,8.3545,6.84316,8.45991,13.3922,-7.63904,5.0377,UCB,10,100
9,1,1,8,5,9,5,6.05705,6.84316,8.45991,13.3922,-7.63904,7.33515,UCB,10,100


In [59]:

show(full_df[(full_df.name.=="UCB").&(full_df.num_agents.==10),:], allrows=true, allcols=true)



1500×15 DataFrame
│ Row  │ run   │ epoch │ agent │ time  │ c     │ state │ reward      │
│      │ Int64 │ Int64 │ Int64 │ Int64 │ Int64 │ Int64 │ Real        │
├──────┼───────┼───────┼───────┼───────┼───────┼───────┼─────────────┤
│ 1    │ 1     │ 1     │ 1     │ 5     │ 10    │ 5     │ -3.42274    │
│ 2    │ 1     │ 1     │ 2     │ 5     │ 10    │ 5     │ -5.72411    │
│ 3    │ 1     │ 1     │ 3     │ 5     │ 10    │ 5     │ -5.2821     │
│ 4    │ 1     │ 1     │ 4     │ 5     │ 10    │ 5     │ -3.07026    │
│ 5    │ 1     │ 1     │ 5     │ 5     │ 10    │ 5     │ -3.26709    │
│ 6    │ 1     │ 1     │ 6     │ 5     │ 10    │ 5     │ -4.29802    │
│ 7    │ 1     │ 1     │ 7     │ 5     │ 10    │ 5     │ -1.34994    │
│ 8    │ 1     │ 1     │ 8     │ 5     │ 10    │ 5     │ -3.51593    │
│ 9    │ 1     │ 1     │ 9     │ 5     │ 9     │ 5     │ -10.9699    │
│ 10   │ 1     │ 1     │ 10    │ 5     │ 9     │ 5     │ -10.9175    │
│ 11   │ 1     │ 2     │ 1     │ 5     │ 8     │ 5     │ -0

│ 159  │ 4     │ 1     │ 9     │ 5     │ 10    │ 5     │ -1.28241    │
│ 160  │ 4     │ 1     │ 10    │ 5     │ 10    │ 5     │ 0.183583    │
│ 161  │ 4     │ 2     │ 1     │ 5     │ 9     │ 5     │ -9.994      │
│ 162  │ 4     │ 2     │ 2     │ 5     │ 9     │ 5     │ -10.5313    │
│ 163  │ 4     │ 2     │ 3     │ 5     │ 9     │ 5     │ -9.28696    │
│ 164  │ 4     │ 2     │ 4     │ 5     │ 9     │ 5     │ -10.21      │
│ 165  │ 4     │ 2     │ 5     │ 5     │ 9     │ 5     │ -11.3377    │
│ 166  │ 4     │ 2     │ 6     │ 5     │ 9     │ 5     │ -10.5522    │
│ 167  │ 4     │ 2     │ 7     │ 5     │ 9     │ 5     │ -10.5626    │
│ 168  │ 4     │ 2     │ 8     │ 5     │ 9     │ 5     │ -9.85432    │
│ 169  │ 4     │ 2     │ 9     │ 5     │ 9     │ 5     │ -10.817     │
│ 170  │ 4     │ 2     │ 10    │ 5     │ 8     │ 5     │ 7.27089     │
│ 171  │ 4     │ 3     │ 1     │ 5     │ 8     │ 5     │ 6.54612     │
│ 172  │ 4     │ 3     │ 2     │ 5     │ 8     │ 5     │ 8.1204      │
│ 173 

│ 302  │ 7     │ 1     │ 2     │ 5     │ 10    │ 5     │ -14.3324    │
│ 303  │ 7     │ 1     │ 3     │ 5     │ 10    │ 5     │ -15.9947    │
│ 304  │ 7     │ 1     │ 4     │ 5     │ 10    │ 5     │ -18.9633    │
│ 305  │ 7     │ 1     │ 5     │ 5     │ 10    │ 5     │ -15.8535    │
│ 306  │ 7     │ 1     │ 6     │ 5     │ 10    │ 5     │ -15.8949    │
│ 307  │ 7     │ 1     │ 7     │ 5     │ 9     │ 5     │ 5.3212      │
│ 308  │ 7     │ 1     │ 8     │ 5     │ 9     │ 5     │ 6.89677     │
│ 309  │ 7     │ 1     │ 9     │ 5     │ 9     │ 5     │ 7.06045     │
│ 310  │ 7     │ 1     │ 10    │ 5     │ 9     │ 5     │ 4.00087     │
│ 311  │ 7     │ 2     │ 1     │ 5     │ 8     │ 5     │ -17.0534    │
│ 312  │ 7     │ 2     │ 2     │ 5     │ 8     │ 5     │ -16.0019    │
│ 313  │ 7     │ 2     │ 3     │ 5     │ 8     │ 5     │ -16.3177    │
│ 314  │ 7     │ 2     │ 4     │ 5     │ 8     │ 5     │ -15.997     │
│ 315  │ 7     │ 2     │ 5     │ 5     │ 8     │ 5     │ -18.4079    │
│ 316 

│ 417  │ 9     │ 2     │ 7     │ 5     │ 7     │ 5     │ 16.4125     │
│ 418  │ 9     │ 2     │ 8     │ 5     │ 7     │ 5     │ 16.8543     │
│ 419  │ 9     │ 2     │ 9     │ 5     │ 7     │ 5     │ 16.8319     │
│ 420  │ 9     │ 2     │ 10    │ 5     │ 7     │ 5     │ 14.1611     │
│ 421  │ 9     │ 3     │ 1     │ 5     │ 6     │ 5     │ 11.7745     │
│ 422  │ 9     │ 3     │ 2     │ 5     │ 6     │ 5     │ 12.091      │
│ 423  │ 9     │ 3     │ 3     │ 5     │ 6     │ 5     │ 12.0168     │
│ 424  │ 9     │ 3     │ 4     │ 5     │ 6     │ 5     │ 12.0004     │
│ 425  │ 9     │ 3     │ 5     │ 5     │ 5     │ 5     │ 0.554836    │
│ 426  │ 9     │ 3     │ 6     │ 5     │ 5     │ 5     │ -0.49509    │
│ 427  │ 9     │ 3     │ 7     │ 5     │ 5     │ 5     │ -2.19955    │
│ 428  │ 9     │ 3     │ 8     │ 5     │ 5     │ 5     │ -3.83107    │
│ 429  │ 9     │ 3     │ 9     │ 5     │ 5     │ 5     │ -1.31933    │
│ 430  │ 9     │ 3     │ 10    │ 5     │ 4     │ 5     │ -8.61198    │
│ 431 

│ 581  │ 12    │ 4     │ 1     │ 5     │ 4     │ 5     │ 14.6199     │
│ 582  │ 12    │ 4     │ 2     │ 5     │ 4     │ 5     │ 13.6153     │
│ 583  │ 12    │ 4     │ 3     │ 5     │ 4     │ 5     │ 17.526      │
│ 584  │ 12    │ 4     │ 4     │ 5     │ 4     │ 5     │ 16.8004     │
│ 585  │ 12    │ 4     │ 5     │ 5     │ 4     │ 5     │ 16.2692     │
│ 586  │ 12    │ 4     │ 6     │ 5     │ 4     │ 5     │ 15.0726     │
│ 587  │ 12    │ 4     │ 7     │ 5     │ 4     │ 5     │ 14.7284     │
│ 588  │ 12    │ 4     │ 8     │ 5     │ 4     │ 5     │ 15.367      │
│ 589  │ 12    │ 4     │ 9     │ 5     │ 3     │ 5     │ -13.462     │
│ 590  │ 12    │ 4     │ 10    │ 5     │ 3     │ 5     │ -10.7359    │
│ 591  │ 12    │ 5     │ 1     │ 5     │ 2     │ 5     │ -13.4131    │
│ 592  │ 12    │ 5     │ 2     │ 5     │ 2     │ 5     │ -12.8965    │
│ 593  │ 12    │ 5     │ 3     │ 5     │ 2     │ 5     │ -13.7174    │
│ 594  │ 12    │ 5     │ 4     │ 5     │ 2     │ 5     │ -13.7284    │
│ 595 

...

│ 99   │ UCB    │ 10         │ 100   │
│ 100  │ UCB    │ 10         │ 100   │
│ 101  │ UCB    │ 10         │ 100   │
│ 102  │ UCB    │ 10         │ 100   │
│ 103  │ UCB    │ 10         │ 100   │
│ 104  │ UCB    │ 10         │ 100   │
│ 105  │ UCB    │ 10         │ 100   │
│ 106  │ UCB    │ 10         │ 100   │
│ 107  │ UCB    │ 10         │ 100   │
│ 108  │ UCB    │ 10         │ 100   │
│ 109  │ UCB    │ 10         │ 100   │
│ 110  │ UCB    │ 10         │ 100   │
│ 111  │ UCB    │ 10         │ 100   │
│ 112  │ UCB    │ 10         │ 100   │
│ 113  │ UCB    │ 10         │ 100   │
│ 114  │ UCB    │ 10         │ 100   │
│ 115  │ UCB    │ 10         │ 100   │
│ 116  │ UCB    │ 10         │ 100   │
│ 117  │ UCB    │ 10         │ 100   │
│ 118  │ UCB    │ 10         │ 100   │
│ 119  │ UCB    │ 10         │ 100   │
│ 120  │ UCB    │ 10         │ 100   │
│ 121  │ UCB    │ 10         │ 100   │
│ 122  │ UCB    │ 10         │ 100   │
│ 123  │ UCB    │ 10         │ 100   │
│ 124  │ UCB    │ 10     

│ 417  │ UCB    │ 10         │ 100   │
│ 418  │ UCB    │ 10         │ 100   │
│ 419  │ UCB    │ 10         │ 100   │
│ 420  │ UCB    │ 10         │ 100   │
│ 421  │ UCB    │ 10         │ 100   │
│ 422  │ UCB    │ 10         │ 100   │
│ 423  │ UCB    │ 10         │ 100   │
│ 424  │ UCB    │ 10         │ 100   │
│ 425  │ UCB    │ 10         │ 100   │
│ 426  │ UCB    │ 10         │ 100   │
│ 427  │ UCB    │ 10         │ 100   │
│ 428  │ UCB    │ 10         │ 100   │
│ 429  │ UCB    │ 10         │ 100   │
│ 430  │ UCB    │ 10         │ 100   │
│ 431  │ UCB    │ 10         │ 100   │
│ 432  │ UCB    │ 10         │ 100   │
│ 433  │ UCB    │ 10         │ 100   │
│ 434  │ UCB    │ 10         │ 100   │
│ 435  │ UCB    │ 10         │ 100   │
│ 436  │ UCB    │ 10         │ 100   │
│ 437  │ UCB    │ 10         │ 100   │
│ 438  │ UCB    │ 10         │ 100   │
│ 439  │ UCB    │ 10         │ 100   │
│ 440  │ UCB    │ 10         │ 100   │
│ 441  │ UCB    │ 10         │ 100   │
│ 442  │ UCB    │ 10     

│ 727  │ UCB    │ 10         │ 100   │
│ 728  │ UCB    │ 10         │ 100   │
│ 729  │ UCB    │ 10         │ 100   │
│ 730  │ UCB    │ 10         │ 100   │
│ 731  │ UCB    │ 10         │ 100   │
│ 732  │ UCB    │ 10         │ 100   │
│ 733  │ UCB    │ 10         │ 100   │
│ 734  │ UCB    │ 10         │ 100   │
│ 735  │ UCB    │ 10         │ 100   │
│ 736  │ UCB    │ 10         │ 100   │
│ 737  │ UCB    │ 10         │ 100   │
│ 738  │ UCB    │ 10         │ 100   │
│ 739  │ UCB    │ 10         │ 100   │
│ 740  │ UCB    │ 10         │ 100   │
│ 741  │ UCB    │ 10         │ 100   │
│ 742  │ UCB    │ 10         │ 100   │
│ 743  │ UCB    │ 10         │ 100   │
│ 744  │ UCB    │ 10         │ 100   │
│ 745  │ UCB    │ 10         │ 100   │
│ 746  │ UCB    │ 10         │ 100   │
│ 747  │ UCB    │ 10         │ 100   │
│ 748  │ UCB    │ 10         │ 100   │
│ 749  │ UCB    │ 10         │ 100   │
│ 750  │ UCB    │ 10         │ 100   │
│ 751  │ UCB    │ 10         │ 100   │
│ 752  │ UCB    │ 10     

│ 1035 │ UCB    │ 10         │ 100   │
│ 1036 │ UCB    │ 10         │ 100   │
│ 1037 │ UCB    │ 10         │ 100   │
│ 1038 │ UCB    │ 10         │ 100   │
│ 1039 │ UCB    │ 10         │ 100   │
│ 1040 │ UCB    │ 10         │ 100   │
│ 1041 │ UCB    │ 10         │ 100   │
│ 1042 │ UCB    │ 10         │ 100   │
│ 1043 │ UCB    │ 10         │ 100   │
│ 1044 │ UCB    │ 10         │ 100   │
│ 1045 │ UCB    │ 10         │ 100   │
│ 1046 │ UCB    │ 10         │ 100   │
│ 1047 │ UCB    │ 10         │ 100   │
│ 1048 │ UCB    │ 10         │ 100   │
│ 1049 │ UCB    │ 10         │ 100   │
│ 1050 │ UCB    │ 10         │ 100   │
│ 1051 │ UCB    │ 10         │ 100   │
│ 1052 │ UCB    │ 10         │ 100   │
│ 1053 │ UCB    │ 10         │ 100   │
│ 1054 │ UCB    │ 10         │ 100   │
│ 1055 │ UCB    │ 10         │ 100   │
│ 1056 │ UCB    │ 10         │ 100   │
│ 1057 │ UCB    │ 10         │ 100   │
│ 1058 │ UCB    │ 10         │ 100   │
│ 1059 │ UCB    │ 10         │ 100   │
│ 1060 │ UCB    │ 10     

│ 1360 │ UCB    │ 10         │ 100   │
│ 1361 │ UCB    │ 10         │ 100   │
│ 1362 │ UCB    │ 10         │ 100   │
│ 1363 │ UCB    │ 10         │ 100   │
│ 1364 │ UCB    │ 10         │ 100   │
│ 1365 │ UCB    │ 10         │ 100   │
│ 1366 │ UCB    │ 10         │ 100   │
│ 1367 │ UCB    │ 10         │ 100   │
│ 1368 │ UCB    │ 10         │ 100   │
│ 1369 │ UCB    │ 10         │ 100   │
│ 1370 │ UCB    │ 10         │ 100   │
│ 1371 │ UCB    │ 10         │ 100   │
│ 1372 │ UCB    │ 10         │ 100   │
│ 1373 │ UCB    │ 10         │ 100   │
│ 1374 │ UCB    │ 10         │ 100   │
│ 1375 │ UCB    │ 10         │ 100   │
│ 1376 │ UCB    │ 10         │ 100   │
│ 1377 │ UCB    │ 10         │ 100   │
│ 1378 │ UCB    │ 10         │ 100   │
│ 1379 │ UCB    │ 10         │ 100   │
│ 1380 │ UCB    │ 10         │ 100   │
│ 1381 │ UCB    │ 10         │ 100   │
│ 1382 │ UCB    │ 10         │ 100   │
│ 1383 │ UCB    │ 10         │ 100   │
│ 1384 │ UCB    │ 10         │ 100   │
│ 1385 │ UCB    │ 10     

In [58]:
mean(full_df[full_df.name.=="UCB",:].Regret)

3.754984072216688